<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="-unSkript-Runbooks-">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="-Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Filter and Terminate EC2 Instances Without Valid Lifetime Tag</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Terminate-EC2-Instances-Without-Valid-Lifetime-Tag"><u>Terminate EC2 Instances Without Valid Lifetime Tag</u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Filter AWS EC2 Instances Without Lifetime Tag</a><br>2)<a href="#2" target="_self" rel="noopener"> Terminate AWS Instance</a></p>

<h3 id="List-all-AWS-Regions"><a id="1" target="_self" rel="nofollow"></a>List all AWS Regions</h3>
<p>This action fetches all AWS Regions to execute Step 1👇. This action will only execute if no <code>region</code> is provided.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>region</code></p>
</blockquote>

In [3]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')

task.configure(outputName="region")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3 id="Filter-AWS-EC2-Instances-Without-Lifetime-Tag"><a id="1" target="_self" rel="nofollow"></a>Filter AWS EC2 Instances Without Lifetime Tag</h3>
<p>Here we will use unSkript Filter AWS EC2 Instances Without Lifetime Tag action to get all the EC2 instances which don't have lifetime tag.</p>
<blockquote>
<p>This action takes the following parameters: <code>lifetime_tag</code>,<code>region</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>untagged_ec2_instances</code></p>
</blockquote>

In [4]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Tuple, Optional
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.utils import CheckOutput, CheckOutputStatus
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint
from datetime import datetime, date

from beartype import beartype
@beartype
def aws_filter_instances_without_termination_and_lifetime_tag_printer(output):
    if output is None:
        return
    if isinstance(output, CheckOutput):
        print(output.json())
    else:
        pprint.pprint(output)

@beartype
def fetch_instances_from_valid_region(res,r):
    result=[]
    instances_dict={}
    for reservation in res:
            for instance in reservation['Instances']:
                try:
                    tagged_instance = instance['Tags']
                    tag_keys = [tags['Key'] for tags in tagged_instance]
                    if 'terminationDateTag' not in tag_keys or 'lifetimeTag' not in tag_keys:
                        result.append(instance['InstanceId'])
                    elif 'terminationDateTag' not in tag_keys and 'lifetimeTag' not in tag_keys:
                        result.append(instance['InstanceId'])
                    if 'terminationDateTag' in tag_keys:
                        for x in instance['Tags']:
                            if x['Key'] == 'terminationDateTag':
                                right_now = date.today()
                                date_object = datetime.strptime(x['Value'], '%d-%m-%Y').date()
                                if date_object < right_now:
                                    result.append(instance['InstanceId'])
                            elif x['Key'] == 'lifetimeTag':
                                launch_time = instance['LaunchTime']
                                convert_to_datetime = launch_time.strftime("%d-%m-%Y")
                                launch_date = datetime.strptime(convert_to_datetime,'%d-%m-%Y').date()
                                if x['Value'] is not 'INDEFINITE':
                                    if launch_date < right_now:
                                        result.append(instance['InstanceId'])
                except Exception as e:
                        if len(instance['InstanceId'])!=0:
                            result.append(instance['InstanceId'])
    if len(result)!=0:
        instances_dict['region']= r
        instances_dict['instances']= result
    return instances_dict

@beartype
def aws_filter_instances_without_termination_and_lifetime_tag(handle, region: str=None) -> CheckOutput:
    """aws_filter_ec2_without_lifetime_tag Returns an List of instances which not have lifetime tag.

        Assumed tag key format - terminationDateTag, lifetimeTag
        Assumed Date format for both keys is -> dd-mm-yy

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Used to filter the instance for specific region.

        :rtype: Object of status, instances which dont having terminationDateTag and lifetimeTag, and error
    """
    final_list=[]
    all_regions = [region]
    if region is None or len(region) == 0:
        all_regions = aws_list_all_regions(handle=handle)
    for r in all_regions:
        try:
            ec2Client = handle.client('ec2', region_name=r)
            all_reservations = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
            instances_without_tags = fetch_instances_from_valid_region(all_reservations,r)
            if len(instances_without_tags)!=0:
                final_list.append(instances_without_tags)
        except Exception as e:
            pass
    if len(final_list)!=0:
        return CheckOutput(status=CheckOutputStatus.FAILED,
                   objects=final_list,
                   error=str(""))
    else:
        return CheckOutput(status=CheckOutputStatus.SUCCESS,
                   objects=final_list,
                   error=str(""))


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(outputName="untagged_ec2_instances")
task.configure(inputParamsJson='''{
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_instances_without_termination_and_lifetime_tag, lego_printer=aws_filter_instances_without_termination_and_lifetime_tag_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Untagged-Instances">Create List of Untagged Instances</h3>
<p>This action filters regions that have no untagged EC2 instances and creates a list of the ones that have have</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_untagged_instances</code></p>
</blockquote>

In [5]:
all_untagged_instances = []
dummy = []
for k,v in untagged_ec2_instances.items():
    if v.status==CheckOutputStatus.FAILED:
        if len(v.objects)!=0:
            dummy = v.objects
            for x in dummy:
                all_untagged_instances.append(x)
print(all_untagged_instances)
task.configure(outputName="all_untagged_instances")

<h3 id="Terminate-AWS-Instance">Terminate AWS Instance</h3>
<p>This action terminates EC2 instances which don't have lifetime tag as captured in Step 1👆</p>
<blockquote>
<p>This action takes the following parameters: <code>instance_ids, region</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>None</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_terminate_instance_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_terminate_instance(handle, instance_ids: List, region: str) -> Dict:
    """aws_terminate_instance Returns an Dict of info terminated instance.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type instance_ids: List
        :param instance_ids: Tag to filter Instances.

        :type region: string
        :param region: Used to filter the instance for specific region.

        :rtype: Dict of info terminated instance.
    """
    ec2Client = handle.client('ec2', region_name=region)
    res = ec2Client.terminate_instances(InstanceIds=instance_ids)

    return res


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "instance_ids": "iter.get(\\"instances\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_untagged_instances",
    "iter_parameter": ["instance_ids","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "if terminate==True",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_terminate_instance, lego_printer=aws_terminate_instance_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions to filter untagged instances and terminate them. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>